In [148]:
import json

In [149]:
# Load the OpenAPI JSON file
with open("doc.json", "r") as f:
    data = json.load(f)


In [150]:
def extract_resource_name(path: str) -> str:
    parts = [part for part in path.split('/') if part]
    return parts[-1] if parts else ""

In [151]:
known_collections = ['users', '_stores']

In [152]:
def pb_field_from_schema(name, schema):
    t = schema.get("type")
    nullable = schema.get("nullable", False)
    max_length = schema.get("maxLength")
    minimum = schema.get("minimum")
    maximum = schema.get("maximum")
    enum = schema.get("enum")
    _required = ''
    if nullable == False:
        _required = ' , Required: true'
    # Skip ID
    if name == "id":
        return None
    
    if enum and isinstance(enum, (list)):
        return f'''&core.SelectField{{Name: "{name}", Values: []string{{"{'","'.join(enum)}"}}{_required}}}'''

    # Handle auto-dates
    if name == "createdAt":
        return f'&core.AutodateField{{Name: "createdAt", OnCreate: true}}'
    if name == "updatedAt":
        return f'&core.AutodateField{{Name: "updatedAt", OnUpdate: true}}'

    # Handle relation fields
    if name.endswith("Id"):
        target = f'{name[:-2]}s'  # strip 'Id'
        if target in known_collections:
            return f'&core.RelationField{{Name: "{name}", CollectionId: {target}Col.Id{_required}}}'
        
    # Handle date fields
    if name.endswith("At"):
        return f'&core.DateField{{Name: "{name}"{_required}}}'

    # Type mappings
    if t == "string":
        if max_length and max_length <= 2048:
            return f'&core.TextField{{Name: "{name}"{_required}}}'
        else:
            return f'&core.TextField{{Name: "{name}"{_required}}}'
    elif t == "boolean":
        return f'&core.BoolField{{Name: "{name}"{_required}}}'
    elif t == "integer" or t == "number":
        min_part = f', Min: types.Pointer[float64]({minimum})' if minimum is not None else ''
        max_part = f', Max: types.Pointer[float64]({maximum})' if maximum is not None else ''
        return f'&core.NumberField{{Name: "{name}"{min_part}{max_part}{_required}}}'
    return None

In [153]:
def extract_relation_fields(properties: dict) -> dict:
    """Return fields matching [name]Id as {field_name: collection_name}."""
    relations = {}
    for field in properties:
        if field.endswith("Id") and field != "id":
            collection = field[:-2]  # remove 'Id' from the end
            relations[field] = f'{collection}s'
    return relations

In [ ]:
# Access the 'paths' object
known_collections = ['users', '_stores']
paths = data.get("paths", {})
# Loop through each path and check for GET methods
i = 2
for m in ["get", "post"]:
    for path, methods in paths.items():
        if path == '/':
            continue
        if m in methods:
            #print(methods)
            #print(f"{m.upper()} path found: {path}")
            # Try to access responses -> 200 -> content -> application/json -> schema -> properties
            try:
                collection = extract_resource_name(path)
                if collection in ['{id}']:
                    path = path.replace('/{id}', '')
                    collection = extract_resource_name(path)
                    print('{id}:', path)
                description = methods[m]["responses"]["200"]["description"]
                tag = methods[m]["tags"][0]
                path = path.replace('/{id}', '')
                _name = '_'.join(path.split('/')).replace('_api', f'collection{i}')
                if _name.find('admin') != -1 and collection in known_collections:
                    continue
                elif _name.find('admin') != -1:
                    _name = _name.replace('_admin', '')
                if collection in ['me', '{id}', 'featured', 'megamenu', '{productId}', '{cartId}', ':cartId', 'create', 'delete', 'login', 'signup'] or collection.find('-') != -1:
                    continue
                if collection in known_collections:
                    continue
                name = _name
                _type = methods[m]["responses"]["200"]["content"]["application/json"]["schema"]["type"]
                _data = properties = methods[m]["responses"]["200"]["content"]["application/json"]["schema"]["properties"].get("data", {}).get("items", {}).get("properties", None)
                if _type == "object" and not _data:
                    properties = methods[m]["responses"]["200"]["content"]["application/json"]["schema"]["properties"]
                else:
                    properties = methods[m]["responses"]["200"]["content"]["application/json"]["schema"]["properties"]["data"]["items"]["properties"]
                relations = extract_relation_fields(properties)
                _rels = ""
                for rel in relations:
                    if relations[rel] in known_collections:
                        _rels += f'{relations[rel]}Col, _ := app.FindCollectionByNameOrId("{relations[rel]}")\n'
                collection_var = collection + "Col"
                code = f'// Migration for collection: {path} {collection} - {tag} - {description}\n'
                code += """package migrations2
    import (
        "github.com/pocketbase/pocketbase/core"
        "github.com/pocketbase/pocketbase/migrations"
        "github.com/pocketbase/pocketbase/tools/types"
    )

    func init() {
        migrations.Register(func(app core.App) error {\n"""
                code += f'\t\t{_rels}\n'
                code += f'\t\t{collection_var}, err := app.FindCollectionByNameOrId("{collection}")\n'
                code += f'\t\tif err != nil {{ {collection_var} = core.NewBaseCollection("{collection}") }}\n'
                code += f'\t\t{collection_var}.ListRule = types.Pointer("")\n'
                code += f'\t\t{collection_var}.ViewRule = types.Pointer("")\n'
                code += f'\t\t{collection_var}.Fields.Add(\n'
                field_lines = []
                for prop_name, prop_schema in properties.items():
                    line = pb_field_from_schema(prop_name, prop_schema)
                    if line:
                        field_lines.append("\t\t\t" + line + "\t\t,")
                code += '\n'.join(field_lines) + '\n)\n'
                code += f'\t\tif err := app.Save({collection_var}); err != nil {{ return err }}\n'
                code += '\t\treturn nil\n'
                code += """\t}, nil)
    }\n"""
                print(f'auto/{name}.go')
                known_collections.append(collection)
                with open(f'auto/{name}.go', mode = "w", encoding = "utf-8") as migrations_file:
                    migrations_file.write(code)
                    migrations_file.close()            
                i += 1
            except Exception as err:
                print(str(err), path)
                pass
                #print(f"GET path {path} has no '200' response with JSON schema properties.")

auto/collection2_vendors.go
{id}: /api/vendors
auto/collection3_categories.go
{id}: /api/categories
{id}: /api/admin/vendors
{id}: /api/admin/categories
auto/collection4_dashboard_summary.go
{id}: /api/admin/users
auto/collection5_products.go
'200' /api/admin/products/create-qr-code
{id}: /api/admin/products
auto/collection6_products_variants.go
auto/collection7_orders.go
{id}: /api/admin/orders
{id}: /api/admin/orders/payment-status
auto/collection8_coupons.go
{id}: /api/admin/coupons
auto/collection9_products_attributes.go
{id}: /api/products
auto/collection10_collections.go
'properties' /api/admin/carts
{id}: /api/carts
auto/collection11_carts.go
auto/collection12_carts_my.go
{id}: /api/carts/refresh
auto/collection13_carts_refresh.go
auto/collection14_chats.go
auto/collection15_reels.go
auto/collection16_orders-public_list.go
{id}: /api/orders
auto/collection17_countries.go
'properties' /api/countries/all
{id}: /api/countries
auto/collection18_currencies.go
auto/collection19_addres